# Comparing Forest Management Scenarios

In [23]:
%matplotlib widget

import __init__
import scripts.config as config
import numpy as np
import pandas as pd
import tempfile
import datetime
import ipywidgets as widgets
from ipywidgets import interact
from natsort import natsorted
import os
from sklearn.metrics import mean_squared_error, r2_score
from matplotlib.font_manager import FontProperties
import seaborn as sns
# import matplotlib as mpl
import matplotlib.pyplot as plt
import importlib
from pathlib import Path
import itertools
from functools import reduce
from statsmodels.iolib.smpickle import load_pickle
import statsmodels.api as sm

In [24]:
# Plotting parameters

XSMALL_SIZE = 6
SMALL_SIZE = 7
MEDIUM_SIZE = 9
BIGGER_SIZE = 12

plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=SMALL_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=SMALL_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('axes', titlesize=SMALL_SIZE)  # fontsize of the figure title
plt.rcParams['figure.dpi'] = 120

This script will visualize the simulation results using all GCMs for all forest management scenarios. All you need to do is include the correct GCM name in the 'gcms' list and scenario names in the 'scenarios' list. These should be identical to the suffixes used in the directories and VELMA XML titles.

In [97]:
gcms = ['canesm2_RCP85', 'ccsm4_RCP85', 'giss_e2_h_RCP85', 'noresm1_m_RCP85']

# Import driver data
sim_start = pd.to_datetime('01-01-2020')
sim_end = pd.to_datetime('12-31-2099')
sim_range = pd.date_range(sim_start, sim_end)

temp_files = []
precip_files = []
for gcm in gcms:
    temp_file_path = config.velma_data / 'temp' / '{}_{}_{}_temp.csv'.format(gcm, 
                                                                             sim_start.year % 100,
                                                                             sim_end.year % 100)
    temp_file = pd.read_csv(temp_file_path, names=['temp'])
    temp_file.index = sim_range
    temp_files.append(temp_file)
    
    precip_file_path = config.velma_data / 'precip' / '{}_{}_{}_ppt.csv'.format(gcm, 
                                                                                sim_start.year % 100, 
                                                                                sim_end.year % 100)
    precip_file = pd.read_csv(precip_file_path, names=['precip'])
    precip_file.index = sim_range
    precip_files.append(precip_files)

In [110]:
# Import results files

# scenarios = ['baseline', 'baseline_water_balance_errors']
# scenarios = ['baseline', 'ind35yr', 'activeall', 'historical']
scenarios = ['baseline', 'ind35yr', 'activeall']

dailies = []
annuals = []

# Import daily and annual results
# Results in nested lists ([x][y], for x management scenarios and y GCMs)
for scenario in scenarios:
    dailies_scenario = []
    annuals_scenario = []
    scenario_dir = config.velma_data.parents[1] / 'results' / scenario
    dirs = os.listdir(scenario_dir)
    for directory in dirs:
        
        results_dir = scenario_dir / 'ellsworth_{}_{}_{}_{}'.format(scenario,
                                                                    sim_start.year % 100,
                                                                    sim_end.year % 100,
                                                                    gcm)

        daily_results = pd.read_csv(results_dir / 'DailyResults.csv')

        # Format datetime
        jday_pad = daily_results['Day'].apply(lambda x: str(x).zfill(3))
        str_year = daily_results['Year'].apply(lambda x: str(x))
        rng = pd.to_datetime((str_year + jday_pad), format='%Y%j')
        daily_results.index = rng
        dailies_scenario.append(daily_results)

#         annual_results = pd.read_csv(results_dir / 'AnnualResults.csv')
#         annuals_scenario.append(annual_results)
        
    dailies.append(dailies_scenario)
#     annuals.append(annuals_scenario)

## Runoff

In [115]:
# Daily runoff time series (all years together)

# # Check that all dfs have same number of columns
# all([len(dailies[0].columns.intersection(df.columns)) 
#       == dailies[0].shape[1] for df in dailies])

# colors = sns.color_palette('tab10', len(sims))

fig, axes = plt.subplots(ncols=1, nrows=4, figsize=(5, 6))

z = [x.groupby(pd.Grouper(freq='w')).sum()['Runoff_All(mm/day)_Delineated_Average'] for x in dailies[0]]
yearly_7day_min = pd.concat([x.groupby(pd.Grouper(freq='y')).min() for x in z], axis=1)
yearly_7day_min.columns = gcms
axes[0].plot(yearly_7day_min, linewidth=0.8)
axes[0].title.set_text('Yearly min 7-day flow')

yearly_7day_max = pd.concat([x.groupby(pd.Grouper(freq='y')).max() for x in z], axis=1)
yearly_7day_max.columns = gcms
axes[1].plot(yearly_7day_max, linewidth=0.8)
axes[1].title.set_text('Yearly max 7-day flow')

z = [x.groupby(pd.Grouper(freq='y')).sum()['Runoff_All(mm/day)_Delineated_Average'] for x in dailies[0]]
yearly_sum = pd.concat(z, axis=1)
yearly_sum.columns = gcms
axes[2].plot(yearly_sum, linewidth=0.8)
axes[2].title.set_text('Yearly sum flow')

z = [x.groupby(pd.Grouper(freq='y')).mean()['Runoff_All(mm/day)_Delineated_Average'] for x in dailies[0]]
yearly_mean = pd.concat(z, axis=1)
yearly_mean.columns = gcms
axes[3].plot(yearly_mean, linewidth=0.8)
axes[3].title.set_text('Yearly mean flow')
    
plt.ylabel('Runoff (mm)')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [145]:
# Averaged GCM flows for all management scenarios
fig, axes = plt.subplots(ncols=2, nrows=4, figsize=(7, 6))
combos = list(itertools.product(gcms, scenarios))

for i, scenario in enumerate(scenarios):
    z = [x.groupby(pd.Grouper(freq='7d')).mean()['Runoff_All(mm/day)_Delineated_Average'] for x in dailies[i]]
    yearly_7day_min = pd.concat([x.groupby(pd.Grouper(freq='y')).min() for x in z], axis=1)
    yearly_7day_min.columns = gcms
    axes[0, 0].plot(yearly_7day_min, linewidth=0.7)
    axes[0, 1].plot(yearly_7day_min.mean(axis=1), linewidth=0.9, label=scenario)
    axes[0, 0].title.set_text('Yearly min 7-day flow')
    axes[0, 1].title.set_text('Yearly min, GCM means')
    axes[0, 0].set_ylim(0, 2)
    axes[0, 1].set_ylim(0, 2)

    yearly_7day_max = pd.concat([x.groupby(pd.Grouper(freq='y')).max() for x in z], axis=1)
    yearly_7day_max.columns = gcms
    axes[1, 0].plot(yearly_7day_max, linewidth=0.7)
    axes[1, 1].plot(yearly_7day_max.mean(axis=1), linewidth=0.9, label=scenario)
    axes[1, 0].title.set_text('Yearly max 7-day flow')
    axes[1, 1].title.set_text('Yearly max, GCM means')

    z = [x.groupby(pd.Grouper(freq='y')).sum()['Runoff_All(mm/day)_Delineated_Average'] for x in dailies[i]]
    yearly_sum = pd.concat(z, axis=1)
    yearly_sum.columns = gcms
    axes[2, 0].plot(yearly_sum, linewidth=0.7)
    axes[2, 1].plot(yearly_sum.mean(axis=1), linewidth=0.9, label=scenario)
    axes[2, 0].title.set_text('Yearly sum flow')
    axes[2, 1].title.set_text('Yearly sum, GCM means')

    z = [x.groupby(pd.Grouper(freq='y')).mean()['Runoff_All(mm/day)_Delineated_Average'] for x in dailies[i]]
    yearly_mean = pd.concat(z, axis=1)
    yearly_mean.columns = gcms
    axes[3, 0].plot(yearly_mean, linewidth=0.7) 
    axes[3, 1].plot(yearly_mean.mean(axis=1), linewidth=0.9, label=scenario)
    axes[3, 0].title.set_text('Yearly mean flow')
    axes[3, 1].title.set_text('Yearly mean, GCM means')

axes[0, 1].legend(loc='upper right', bbox_to_anchor=(1, 1.5), fancybox=True, ncol=3)
plt.ylabel('Runoff (mm)')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Stream temperature

In [126]:
# Get cell writer files

cell_results = []
for scenario in scenarios:
    cell_results_scenario = []
    scenario_dir = config.velma_data.parents[1] / 'results' / scenario
    dirs = os.listdir(scenario_dir)
    for directory in dirs:
        cell_paths = []
        results_dir = scenario_dir / 'ellsworth_{}_{}_{}_{}'.format(scenario,
                                                                    sim_start.year % 100,
                                                                    sim_end.year % 100,
                                                                    gcm)
        for file in os.listdir(results_dir):
            if file.startswith('Cell_'):
                cell_paths.append(file)
        
        nodes = []
        for path in cell_paths:
            nodes.append(path.split('_')[-1])
        
        cell_paths_sorted = [x for _,x in natsorted(zip(nodes,cell_paths))]
        
        for path in [cell_paths_sorted[0]]:  # Only need the first cell, which is the Ellsworth mouth/outlet
            cell_result = pd.read_csv(results_dir / path)
            jday_pad = cell_result['Jday'].apply(lambda x: str(x).zfill(3))
            str_year = cell_result['Year'].apply(lambda x: str(x))
            cell_result['date'] = str_year + jday_pad
            rng = pd.to_datetime(cell_result['date'], format='%Y%j')
            cell_result.index = rng
            cell_results_scenario.append(cell_result)
    
    cell_results.append(cell_results_scenario)

In [127]:
# Correct VELMA stream temperature seasonal bias using pre-trained regression model
# *** Not sure if this correction is still valid considering the non-linear seasonal changes of the climate projections ***

olsmodel = load_pickle(config.data_path.parents[0] / 'models' / 'stream_temp_correction_ols.pickle')

stream_temps_corrected = []
for i, scenario in enumerate(scenarios):
    stream_temps_scenario = []
    for j, gcm in enumerate(gcms):
        z = cell_results[i][j]['Water_Surface_Temperature(degrees_C)']
        
        day = 24 * 60 * 60
        year = 365.2425 * day
        timestamp_secs = pd.to_datetime(z.index)
        timestamp_secs = timestamp_secs.map(datetime.datetime.timestamp)
        year_cos = np.cos(timestamp_secs * (2 * np.pi / year))
        year_sin = np.sin(timestamp_secs * (2 * np.pi / year))

        y = pd.DataFrame(data=np.column_stack([z, year_cos, year_sin]), columns=['temp', 'year_cos', 'year_sin'])
        y.index = z.index
        y['air_temp_3day_avg'] = y['temp'].rolling(3, min_periods=0).mean()

        y = sm.add_constant(y)
        y['streamtemp_corrected'] = olsmodel.predict(y)
        
        stream_temps_scenario.append(y['streamtemp_corrected'])
    
    stream_temps_corrected.append(stream_temps_scenario)

In [146]:
plt.close('all')
fig, axes = plt.subplots(ncols=2, nrows=3, figsize=(7, 5))

for i, scenario in enumerate(scenarios):
    z = [x for x in stream_temps_corrected[i]]
    stream_temp = pd.concat([x.groupby(pd.Grouper(freq='y')).mean() for x in z], axis=1)
    stream_temp.columns = gcms
    axes[0, 0].plot(stream_temp, linewidth=0.7, label=scenario)
    axes[0, 0].title.set_text('Yearly mean')
    axes[0, 1].plot(stream_temp.mean(axis=1), label=scenario)
    axes[0, 1].title.set_text('Yearly mean, GCM means')
    
    z = [x.groupby(pd.Grouper(freq='7d')).mean() for x in stream_temps_corrected[i]]
    stream_temp = pd.concat([x.groupby(pd.Grouper(freq='y')).min() for x in z], axis=1)
    stream_temp.columns = gcms
    axes[1, 0].plot(stream_temp, linewidth=0.7, label=scenario)
    axes[1, 0].title.set_text('Yearly min 7-day average')
    axes[1, 1].plot(stream_temp.mean(axis=1), label=scenario)
    axes[1, 1].title.set_text('Yearly min 7-day average, GCM means')   
    
    z = [x.groupby(pd.Grouper(freq='7d')).mean() for x in stream_temps_corrected[i]]
    stream_temp = pd.concat([x.groupby(pd.Grouper(freq='y')).max() for x in z], axis=1)
    stream_temp.columns = gcms
    axes[2, 0].plot(stream_temp, linewidth=0.7, label=scenario)
    axes[2, 0].title.set_text('Yearly max 7-day average')
    axes[2, 1].plot(stream_temp.mean(axis=1), label=scenario)
    axes[2, 1].title.set_text('Yearly max 7-day average, GCM means')   

axes[0, 1].legend(loc='upper right', bbox_to_anchor=(1, 1.5), fancybox=True, ncol=3)
plt.ylabel('Temperature (C)')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Stream chemistry

In [148]:
fig, axes = plt.subplots(ncols=2, nrows=4, figsize=(6.5, 5.5))

for i, scenario in enumerate(scenarios):
    z = [x[['NH4(gN/m2)_Layer1', 'NH4(gN/m2)_Layer2', 'NH4(gN/m2)_Layer3', 'NH4(gN/m2)_Layer4']].sum(axis=1) for x in cell_results[i]]
    y = [x.groupby(pd.Grouper(freq='y')).mean() for x in z]
    nh4 = pd.concat(y, axis=1)
    nh4.columns = gcms
    axes[0, 0].plot(nh4, linewidth=0.7)
    axes[0, 1].plot(nh4.mean(axis=1), label=scenario)
    axes[0, 0].title.set_text('NH4')
    axes[0, 1].title.set_text('NH4, GCM means')

    z = [x[['NO3(gN/m2)_Layer1', 'NO3(gN/m2)_Layer2', 'NO3(gN/m2)_Layer3', 'NO3(gN/m2)_Layer4']].sum(axis=1) for x in cell_results[i]]
    y = [x.groupby(pd.Grouper(freq='y')).mean() for x in z]
    no3 = pd.concat(y, axis=1)
    no3.columns = gcms
    axes[1, 0].plot(no3, linewidth=0.7)
    axes[1, 1].plot(no3.mean(axis=1), label=scenario)
    axes[1, 0].title.set_text('NO3')
    axes[1, 1].title.set_text('NO3, GCM means')
    
    z = [x[['DON(gN/m2)_Layer1', 'DON(gN/m2)_Layer2', 'DON(gN/m2)_Layer3', 'DON(gN/m2)_Layer4']].sum(axis=1) for x in cell_results[i]]
    y = [x.groupby(pd.Grouper(freq='y')).mean() for x in z]
    don = pd.concat(y, axis=1)
    don.columns = gcms
    axes[2, 0].plot(don, linewidth=0.7)
    axes[2, 1].plot(don.mean(axis=1), label=scenario)
    axes[2, 0].title.set_text('DON')
    axes[2, 1].title.set_text('DON, GCM means')

    z = [x[['DOC(gC/m2)_Layer1', 'DOC(gC/m2)_Layer2', 'DOC(gC/m2)_Layer3', 'DOC(gC/m2)_Layer4']].sum(axis=1) for x in cell_results[i]]
    y = [x.groupby(pd.Grouper(freq='y')).mean() for x in z]
    doc = pd.concat(y, axis=1)
    doc.columns = gcms
    axes[3, 0].plot(doc, linewidth=0.7) 
    axes[3, 1].plot(doc.mean(axis=1), label=scenario)
    axes[3, 0].title.set_text('DOC')
    axes[3, 1].title.set_text('DOC, GCM means')

axes[0, 1].legend(loc='upper right', bbox_to_anchor=(1, 1.6), fancybox=True, ncol=3)
plt.ylabel('g/m2')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Carbon

In [149]:
fig, axes = plt.subplots(ncols=2, nrows=5, figsize=(8, 7))

for i, scenario in enumerate(scenarios):
    z = [x['agBiomass_Pool(gC/m2)_Delineated_Average'] for x in dailies[i]]
    ag_biomass_pool = pd.concat(z, axis=1)
    ag_biomass_pool.columns = gcms
    axes[0, 0].plot(ag_biomass_pool, linewidth=0.7)
    axes[0, 1].plot(ag_biomass_pool.mean(axis=1), label=scenario)
    axes[0, 0].title.set_text('agBiomass Pool')
    axes[0, 1].title.set_text('GCM Means')
 
    z = [x['bgBiomass_Pool(gC/m2)_Delineated_Average'] for x in dailies[i]]
    bg_biomass_pool = pd.concat(z, axis=1)
    bg_biomass_pool.columns = gcms
    axes[1, 0].plot(bg_biomass_pool, linewidth=0.7)
    axes[1, 1].plot(bg_biomass_pool.mean(axis=1), label=scenario)
    axes[1, 0].title.set_text('bgBiomass Pool')

    z = [x['agLitter_Pool(gC/m2)_Delineated_Average'] for x in dailies[i]]
    ag_litter_pool = pd.concat(z, axis=1)
    ag_litter_pool.columns = gcms
    axes[2, 0].plot(ag_litter_pool, linewidth=0.7)
    axes[2, 1].plot(ag_litter_pool.mean(axis=1), label=scenario)
    axes[2, 0].title.set_text('agLitter Pool')

    z = [x['bgLitter_Pool(gC/m2)_Delineated_Average'] for x in dailies[i]]
    bg_litter_pool = pd.concat(z, axis=1)
    bg_litter_pool.columns = gcms
    axes[3, 0].plot(bg_litter_pool, linewidth=0.7) 
    axes[3, 1].plot(bg_litter_pool.mean(axis=1), label=scenario)
    axes[3, 0].title.set_text('agLitter Pool')
    
    z = [x['Humus_Pool(gC/m2)_Delineated_Average'] for x in dailies[i]]
    humus_pool = pd.concat(z, axis=1)
    humus_pool.columns = gcms
    axes[4, 0].plot(humus_pool, linewidth=0.7) 
    axes[4, 1].plot(humus_pool.mean(axis=1), label=scenario)
    axes[4, 0].title.set_text('Humus Pool')
    
axes[0, 1].legend(loc='upper right', bbox_to_anchor=(1, 1.6), fancybox=True, ncol=3)
plt.ylabel('gC/m2')
plt.tight_layout(rect=[0, 0, 1, 0.99])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
soil = np.loadtxt(config.velma_data / 'soil' / 'MapunitRaster_10m.asc', dtype='int', delimiter=',', skiprows=6)
plt.figure()
plt.imshow(soil)
np.unique(soil)
soil.dtype
# soil_int = soil.astype('int')
# soil_int

from soil.soil_merger import readHeader
header = readHeader(config.velma_data / 'soil' / 'MapunitRaster_10m.asc')
outfile = config.velma_data / 'soil' / 'MapunitRaster_10m.asc'
f = open(outfile, 'w')
f.write(header)
np.savetxt(f, soil_int.astype(int), fmt='%i')
f.close()

ValueError: could not convert string to float: '10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 4 4 4 4 4 4 4 4 4 4 4 4 4 4 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10 10